In [1]:
# RAGデータ投入処理

In [9]:
# コレクションの削除
client.delete_collection("daily_docs")  # コレクション名を指定
client.delete_collection("weekly_docs")  # コレクション名を指定
client.delete_collection("monthly_docs")  # コレクション名を指定

In [6]:
# ✅ PDF のテキスト抽出・クリーニング・要約
def extract_text_from_pdf(pdf_path, use_summary=True):
    """PDFからマーケット情報を抽出し、クリーニング後にGeminiで要約"""
    reader = PdfReader(pdf_path)
    raw_text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    
    # ✅ クリーニング処理
    clean_text = clean_pdf_text(raw_text)
    
    if use_summary:
        summary = summarize_with_gemini(clean_text)
        return summary if summary else clean_text  # 要約が失敗した場合はクリーニング済みテキストを返す

    return clean_text  # 要約を使わない場合はクリーンテキストのみ

def clean_pdf_text(text):
    """抽出したテキストを整形して不要な部分を削除"""

    # ✅ ページ番号やURLの削除
    text = re.sub(r"\s*[-_]+\s*\d+/\d+\s*[-_]+", "", text)  # "1/3", "2/3" などのページ表記
    text = re.sub(r"https?://\S+", "", text)  # URLを削除

    # ✅ 免責事項・注意書きを削除
    exclusion_patterns = [
        r"本資料に関してご留意頂きたい事項.*$",
        r"出所）.*?$",
        r"本資料は.*?作成されたものです。",
        r"本資料は、作成時点 で.*$",
        r"本資料の内容は.*?変更されることがあります。",
        r"本資料は.*?保証するものではありません。",
        r"本資料に示す意見等は.*?限りません。",
        r"本資料中で使用している指数について.*$",
    ]
    for pattern in exclusion_patterns:
        text = re.sub(pattern, "", text, flags=re.MULTILINE)

    # ✅ 連続する空白・改行の削除
    text = re.sub(r"\n\s*\n", "\n", text)  # 空行を削除
    text = re.sub(r"[ \t]+", " ", text)  # 連続する空白を1つに
    text = text.strip()

    return text

def summarize_with_gemini(text):
    """Gemini に問い合わせてテキストの要約を取得"""
    prompt = f"""
    以下のマーケット情報の要点を簡潔に要約してください：
    
    {text}
    
    観点：[1. 主要金融市場の動き, 2. 主要国株式の動き, 3. マーケットの動き, 4. 注目点]
    なお、免責事項などの記載は無駄な文章なので除外して。
    """
    try:
        gemini_model = genai.GenerativeModel("gemini-1.5-flash")
        response = gemini_model.generate_content(prompt)
        if hasattr(response, "text") and response.text:
            return response.text.strip().replace("\n"," ").replace("*","")
        else:
            print(f"⚠️ 要約取得失敗: {response}")
            return None
    except Exception as e:
        print(f"❌ Gemini 要約エラー: {str(e)}")
        return None

In [10]:
# データ投入用関数群
import os
import pdfplumber
import chromadb
import google.generativeai as genai
import uuid
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
from PyPDF2 import PdfReader

# ✅ 環境変数からAPIキーを設定
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ✅ ChromaDB の HTTP クライアント（Docker コンテナの chromadb に接続）
client = chromadb.HttpClient(host="chromadb", port=8000)

# ✅ コレクション作成
collections = {
    "daily": client.get_or_create_collection("daily_docs"),
    "weekly": client.get_or_create_collection("weekly_docs"),
    "monthly": client.get_or_create_collection("monthly_docs"),
}

# ✅ 適切な埋め込みモデルを利用
EMBEDDING_MODEL = "models/text-embedding-004"  # 高精度な768次元埋め込み

def get_gemini_embedding(text):
    """Gemini APIを使用してテキストの埋め込みを取得"""
    try:
        response = genai.embed_content(model=EMBEDDING_MODEL, content=text)
        if "embedding" in response:
            return response["embedding"]
        else:
            print(f"⚠️ 埋め込み取得失敗（レスポンス不正）: {response}")
            return None
    except Exception as e:
        print(f"❌ 埋め込みエラー: {str(e)}")
        return None

# ✅ チャンク化（トークンベースで分割 & オーバーラップ設定）
def split_text(text, chunk_size=300, overlap=50):
    """テキストを適切なチャンクサイズで分割"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=overlap, length_function=len
    )
    return text_splitter.split_text(text)

# ✅ PDF のテキストを用途別に登録（チャンク分割対応 & Gemini要約適用）
def process_pdf(file_path, category):
    """PDFファイルを読み込んでGeminiで要約し、ChromaDBへ登録（チャンク分割対応）"""
    if category not in collections:
        print(f"⚠️ 未知のカテゴリ '{category}' を 'other' に変更")
        category = "other"

    # ✅ PDF からクリーニングしたテキストを取得
    summarized_text = extract_text_from_pdf(file_path, use_summary=True)
    
    if not summarized_text.strip():
        print(f"⚠️ {file_path} はテキストを抽出できませんでした")
        return
    
    # ✅ トークンベースで分割
    text_chunks = split_text(summarized_text)

    for idx, chunk in enumerate(text_chunks):
        embedding = get_gemini_embedding(chunk)
        if embedding is None:
            print(f"⚠️ 埋め込み失敗: チャンク {idx+1}/{len(text_chunks)} (ファイル: {file_path})")
            continue

        doc_id = str(uuid.uuid4())  # ユニークなIDを生成

        try:
            collections[category].add(
                ids=[doc_id],
                documents=[chunk],
                metadatas=[{"source": file_path, "category": category, "chunk_index": idx}],
                embeddings=[embedding]
            )
            print(f"✅ チャンク {idx+1}/{len(text_chunks)} 登録完了: {file_path} → {category} (ID: {doc_id})")

        except Exception as e:
            print(f"❌ ChromaDB 登録エラー: {str(e)}")



In [11]:
# ✅ テスト用PDFの登録
INPUT_DIR = "input/data-rag/"
pdf_files = {
    INPUT_DIR + "daily250221.pdf": "daily",
    INPUT_DIR + "daily250220.pdf": "daily",
    INPUT_DIR + "daily250219.pdf": "daily",
    INPUT_DIR + "daily250218.pdf": "daily",
    INPUT_DIR + "250217_weekly.pdf": "weekly",
    INPUT_DIR + "250210_weekly.pdf": "weekly",
    INPUT_DIR + "250203_weekly.pdf": "weekly",
    INPUT_DIR + "monthly_2502.pdf": "monthly",
}

for file, category in pdf_files.items():
    process_pdf(file, category)

✅ チャンク 1/1 登録完了: input/data-rag/daily250221.pdf → daily (ID: 0e27188c-b8a7-4a95-a69e-3e4215eba9eb)
✅ チャンク 1/1 登録完了: input/data-rag/daily250220.pdf → daily (ID: 0b5e0c7e-fa22-4f33-bd28-0cc7e2ab2c62)
✅ チャンク 1/1 登録完了: input/data-rag/daily250219.pdf → daily (ID: e383c4ca-a80a-4a57-a07d-d6971f296b1d)
✅ チャンク 1/1 登録完了: input/data-rag/daily250218.pdf → daily (ID: af4e8b7d-1138-46a4-93b5-0657c5147933)
✅ チャンク 1/3 登録完了: input/data-rag/250217_weekly.pdf → weekly (ID: 9ce63654-8505-4dca-b721-e2bdc28a71d1)
✅ チャンク 2/3 登録完了: input/data-rag/250217_weekly.pdf → weekly (ID: 576a02c2-12a2-44e8-8f47-08ceaca57de6)
✅ チャンク 3/3 登録完了: input/data-rag/250217_weekly.pdf → weekly (ID: e2ddb008-d099-4f13-b8e1-72ed40ddcde9)
✅ チャンク 1/3 登録完了: input/data-rag/250210_weekly.pdf → weekly (ID: 255cc25f-cf1d-4d4b-92a7-fbae1128328f)
✅ チャンク 2/3 登録完了: input/data-rag/250210_weekly.pdf → weekly (ID: 57930c25-5393-4320-ab8f-fcae6fa9d1db)
✅ チャンク 3/3 登録完了: input/data-rag/250210_weekly.pdf → weekly (ID: 9f60a0d2-a83e-4991-a29a-59de7c

In [12]:
# RAGへ問い合わせ用関数群
# ✅ ChromaDB 内のデータを確認
def check_chromadb_data():
    """ChromaDB にデータが正しく登録されているか確認"""
    for category, collection in collections.items():
        try:
            data = collection.peek()
            print(f"📌 {category} コレクションのデータ:")
            print(data)
        except Exception as e:
            print(f"⚠️ コレクション {category} のデータ確認エラー: {str(e)}")

# ✅ クエリのリライト（LLMを利用）
def rewrite_query(query):
    """LLM を使ってクエリを拡張"""
    prompt = f"""
    ユーザーのクエリをより具体的に拡張してください。
    
    クエリ: {query}
    
    出力例: 2025年2月の金融市場動向と米国株式市場の関係
    """
    try:
        gemini_model = genai.GenerativeModel("gemini-1.5-flash")
        response = gemini_model.generate_content(prompt)
        if hasattr(response, "text") and response.text:
            print(f"🟢 クエリ拡張成功: {response.text}")
            return response.text.strip()
        else:
            print(f"⚠️ クエリ拡張失敗（レスポンス不正）: {response}")
            return query
    except Exception as e:
        print(f"❌ クエリ拡張エラー: {str(e)}")
        return query

# ✅ ChromaDB に問い合わせ（類似度フィルタ適用）
def retrieve_relevant_info(query: str, top_k=3, min_score=0.1):
    """ChromaDB から類似情報を取得（類似度閾値なし）"""
    expanded_query = rewrite_query(query)  # クエリ拡張
    print(f"🟢 クエリ拡張後: {expanded_query}")

    query_embedding = get_gemini_embedding(expanded_query)  # ✅ クエリをベクトル化
    if query_embedding is None:
        print("❌ クエリ埋め込みの取得に失敗しました")
        return "🔍 クエリの埋め込みに失敗しました。"

    # ✅ クエリ埋め込みの次元確認
    print(f"✅ クエリの埋め込みベクトルの次元: {len(query_embedding)}")

    # ✅ ChromaDB に格納されているデータの埋め込み次元確認
    peek_data = collections["daily"].peek()
    if "embeddings" in peek_data and len(peek_data["embeddings"]) > 0:
        print(f"✅ 格納済みデータの埋め込みベクトルの次元: {len(peek_data['embeddings'][0])}")
    
    try:
        results = collections["daily"].query(
            query_embeddings=[query_embedding],
            n_results=top_k,
            include=["documents", "metadatas", "distances"]
        )
    except Exception as e:
        print(f"❌ クエリ実行エラー: {str(e)}")
        return "🔍 クエリの実行に失敗しました。"

    if not results.get("documents"):
        return "🔍 関連情報なし"

    retrieved_docs = []
    for i in range(len(results["documents"][0])):
        score = results["distances"][0][i]
        if score < min_score:
            continue  # スコアが低すぎる場合は無視

        retrieved_docs.append(f"🔹 類似情報 {i+1}: {results['documents'][0][i][:200]}...")
        retrieved_docs.append(f"📝 メタデータ: {results['metadatas'][0][i]}")
        retrieved_docs.append(f"🔢 類似度スコア: {score:.4f}")
        retrieved_docs.append("-" * 50)

    return "\n".join(retrieved_docs) if retrieved_docs else "🔍 適切な情報が見つかりませんでした。"
    for i in range(len(results["documents"][0])):
        retrieved_docs.append(f"🔹 類似情報 {i+1}: {results['documents'][0][i][:200]}...")
        retrieved_docs.append(f"📝 メタデータ: {results['metadatas'][0][i]}")
        retrieved_docs.append(f"🔢 類似度スコア: {results['distances'][0][i]:.4f}")
        retrieved_docs.append("-" * 50)

    return "\n".join(retrieved_docs) if retrieved_docs else "🔍 適切な情報が見つかりませんでした。"

# ✅ LLM に検索結果を要約させる
def summarize_retrieved_info(query: str, top_k=3):
    """RAG 検索結果を LLM で要約"""
    retrieved_docs = retrieve_relevant_info(query, top_k)
    
    if "🔍 適切な情報が見つかりませんでした。" in retrieved_docs:
        print("❌ 要約不可: 適切な検索結果がありません。")
        return "🔍 要約できる情報が見つかりませんでした。"

    summary_prompt = f"""
    あなたは金融市場の専門家です。
    以下の情報を要約し、最近の金融市場の動向について詳しく解説してください。

    ### 🔍 検索結果:
    {retrieved_docs}

    ---
    
    要約:
    """
    try:
        gemini_model = genai.GenerativeModel("gemini-1.5-flash")
        response = gemini_model.generate_content(summary_prompt)
        
        if hasattr(response, "text") and response.text:
            print(f"📝 要約成功: {response.text.strip()}")
            return response.text.strip()
        else:
            print(f"⚠️ 要約失敗（レスポンス不正）: {response}")
            return "🔍 要約生成に失敗しました。"

    except Exception as e:
        print(f"❌ 要約エラー: {str(e)}")
        return "🔍 要約の生成に失敗しました。"

In [13]:
# ✅ 検証用コード
if __name__ == "__main__":
    check_chromadb_data()  # ChromaDB にデータが登録されているか確認

    # クエリの実行と LLM での要約
    query_text = "マーケットの動きとして日本動向を教えて"
    summary_info = summarize_retrieved_info(query_text, top_k=3)

    print("\n=== 🔥 LLM による要約結果 ===")
    print(summary_info)


📌 daily コレクションのデータ:
{'ids': ['0e27188c-b8a7-4a95-a69e-3e4215eba9eb', '0b5e0c7e-fa22-4f33-bd28-0cc7e2ab2c62', 'e383c4ca-a80a-4a57-a07d-d6971f296b1d', 'af4e8b7d-1138-46a4-93b5-0657c5147933'], 'embeddings': array([[ 0.00895457,  0.02992844,  0.01273694, ..., -0.04591253,
         0.05165695, -0.0279527 ],
       [-0.00369174,  0.02553232,  0.00575919, ..., -0.03026854,
         0.05744867, -0.01060821],
       [ 0.01338212,  0.02822397, -0.02249661, ..., -0.01776579,
         0.04947588, -0.01638838],
       [ 0.02455232,  0.03504896, -0.03395839, ..., -0.02619648,
         0.04096452, -0.02295619]]), 'metadatas': [{'category': 'daily', 'chunk_index': 0, 'source': 'input/data-rag/daily250221.pdf'}, {'category': 'daily', 'chunk_index': 0, 'source': 'input/data-rag/daily250220.pdf'}, {'category': 'daily', 'chunk_index': 0, 'source': 'input/data-rag/daily250219.pdf'}, {'category': 'daily', 'chunk_index': 0, 'source': 'input/data-rag/daily250218.pdf'}], 'documents': ['2月20日、主要市場は概ね下落。日経平均株価は4

In [14]:
!pip list

Package                                  Version
---------------------------------------- ------------
aiohappyeyeballs                         2.4.6
aiohttp                                  3.11.13
aiosignal                                1.3.2
alembic                                  1.12.0
altair                                   5.1.2
annotated-types                          0.7.0
anyio                                    4.0.0
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
arrow                                    1.3.0
asgiref                                  3.8.1
asttokens                                2.4.0
async-generator                          1.10
async-lru                                2.0.4
attrs                                    23.1.0
Babel                                    2.13.0
backcall                                 0.2.0
backoff                                  2.2.1
backports.functools-lru-cache            1.6.

In [1]:
!pip list

Package                                  Version
---------------------------------------- ------------
alembic                                  1.12.0
altair                                   5.1.2
annotated-types                          0.7.0
anyio                                    4.0.0
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
arrow                                    1.3.0
asgiref                                  3.8.1
asttokens                                2.4.0
async-generator                          1.10
async-lru                                2.0.4
attrs                                    23.1.0
Babel                                    2.13.0
backcall                                 0.2.0
backoff                                  2.2.1
backports.functools-lru-cache            1.6.5
bcrypt                                   4.2.1
beautifulsoup4                           4.12.2
bleach                                   6.1.0